In [1]:
import os
import sys
import re

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# list shp files recursively
os.chdir('..')
abs_path = os.getcwd()

HIFLD_path = os.path.join(abs_path, 'data/HIFLD/centroids')
shp_files = [os.path.join(root, name) \
             for root, dirs, files in os.walk(HIFLD_path) \
             for name in files \
             if name.endswith(('.shp'))]

tribal_WA_folder = os.path.join(abs_path, 'data/Tribal_Lands_WA')
tribal_WA_file = 'TribalLands.shp'
tribal_WA_path = os.path.join(tribal_WA_folder, tribal_WA_file)

In [3]:
def read_shp(file, rows=100):
    df = gpd.read_file(file, rows=rows)

    return df

In [4]:
# read files
tribal_WA_shapes = gpd.read_file(tribal_WA_path)
# for all shp_files
centroids_dict = dict()
for file in shp_files:
    basename = os.path.basename(file).split('/')[0]
    fname = os.path.basename(basename).split('.')[0]

    # print(file)
    df = read_shp(file, rows=500)
    centroids_dict[fname] = df
    # df_address[fname]['Facility_type'] = fname

In [5]:
# convert to CRS 'EPSG:4326'

def convert_EPSG4326(dict):
    """
    dict from full_address
    maybe want to store source data before conversion
    """
    for fname in dict:
        dict[fname] = dict[fname].to_crs("EPSG:4326")

    return dict

In [6]:
dict_EPSG4326 = convert_EPSG4326(centroids_dict)
tribal_WA_shapes = tribal_WA_shapes.to_crs("EPSG:4326")

# Spatial Joins

In [7]:
def spatial_join(dict, tribal, how='left'):
    """
    Check if coordinates falls within tribal lands
    Make new column 'Tribal'
    """
    dict_sjoin = {}
    for fname in dict:
        df = dict[fname].sjoin(tribal, how=how)
        
        # if centroid in tribal polygon, label as 1
        df['Tribal'] = df.index_right.apply(lambda x: 0 if pd.isna(x) else 1) 
        dict_sjoin[fname] = df        

    return dict_sjoin    

In [8]:
dict_sjoin = spatial_join(dict_EPSG4326, tribal_WA_shapes)

In [9]:
for fname in dict_sjoin:
    print(fname)

AllPlacesOfWorship
FDIC_Insured_Banks
Fire_Stations
Prison_Boundaries
PublicSchools
UrgentCareFacs


In [10]:
dict_sjoin['PublicSchools']

,Full_Addre,Place_type,source_lon,source_lat,geometry,index_right,OBJECTID,TRIBAL_NM,TRIBAL_NM1,TRIBAL_NM2,...,OLD_RES_NM,WEBLINK,GlobalID,created_us,created_da,last_edite,last_edi_1,SHAPE_Leng,SHAPE_Area,Tribal
0,"9604 ILLINOIS ST, HEBRON, IL 60034",PublicSchools,-88.431010,42.465566,POINT (-88.43101 42.46557),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,"4420 DENVER AVE, CHARLOTTE, NC 28208",PublicSchools,-80.911501,35.231909,POINT (-80.91150 35.23191),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,"965 GRIZZLY CUB DR, FRANKLIN, IN 46131",PublicSchools,-86.062430,39.489753,POINT (-86.06243 39.48975),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,"22700 S POWER RD, QUEEN CREEK, AZ 85142",PublicSchools,-111.687306,33.242888,POINT (-111.68731 33.24289),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,"24901 S POWER RD, QUEEN CREEK, AZ 85142",PublicSchools,-111.683768,33.222170,POINT (-111.68377 33.22217),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"7201 BEACON AVE S, SEATTLE, WA 98108",PublicSchools,-122.296327,47.538394,POINT (-122.29633 47.53839),38.0,39.0,None,None,None,...,None,http://files.usgwarchives.net/wa/indians/treat...,{F48544E9-67EE-4EC7-B3D8-EB361A1B42B7},WAECY_Geoservices,2023-10-13,WAECY_Geoservices,2023-10-13,4.958401e+06,-2.821400e+11,1
496,"55 NORTH SALIDA WAY, AURORA, CO 80011",PublicSchools,-104.787063,39.717726,POINT (-104.78706 39.71773),NaN,NaN,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
497,"1401 WEST FREMONT, SELAH, WA 98942",PublicSchools,-120.548284,46.657102,POINT (-120.54828 46.65710),30.0,31.0,None,None,None,...,None,http://files.usgwarchives.net/wa/indians/treat...,{6EA4E8D9-57C8-4404-AAD7-B957BF5C44A2},WAECY_Geoservices,2023-10-13,WAECY_Geoservices,2023-10-13,6.846421e+06,4.489107e+11,1
498,"411 NORTH FIRST STREET, SELAH, WA 98942",PublicSchools,-120.527819,46.659899,POINT (-120.52782 46.65990),30.0,31.0,None,None,None,...,None,http://files.usgwarchives.net/wa/indians/treat...,{6EA4E8D9-57C8-4404-AAD7-B957BF5C44A2},WAECY_Geoservices,2023-10-13,WAECY_Geoservices,2023-10-13,6.846421e+06,4.489107e+11,1


In [11]:
def save_shp(dict, save_dir):
    for fname in dict:
        # dict[fname]['source_centroid'] = gpd.GeoSeries.from_wkt(dict[fname]['source_centroid'])
        shp_file = dict[fname].set_geometry('geometry')
        # shp_file = gpd.GeoDataFrame(dict[fname], geometry=dict[fname]['source_centroid'])
        # shp_file.to_file(os.path.join(save_dir, ('{}.shp'.format(fname))), driver='ESRI Shapefile')
        save_path = os.path.join(save_dir, f"{fname}")
        create_dir(save_path)
        shp_file.to_file(save_path, driver='ESRI Shapefile')

def create_dir(save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

In [13]:
save_dir = os.path.join(abs_path, 'data/HIFLD/spatial_join')
create_dir(save_dir)

save_shp(dict_sjoin, save_dir)

C:\Users\bchan\AppData\Local\Temp\ipykernel_8784\177911819.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_8784\177911819.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_8784\177911819.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_8784\177911819.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_file.to_file(save_path, driver='ESRI Shapefile')
C:\Users\bchan\AppData\Local\Temp\ipykernel_8784\177911819.py:9: UserWarning: Column names longer than 10 characters wil